Our goal is to merge three datasets , one extracted from the documents and the second is scrapped from wikipedia using the species extracted from the documents are keywords and the third one is using the species scrapped from the plant list as keywords

In [1]:
import pandas as pd

In [2]:
data1 = pd.read_csv('../generetad data/scrapped_wikidata.csv')
data2=pd.read_csv('../generetad data/species_and_titles_dataframe.csv')

In [3]:
data1

,espèce,description
0,Schefflera abyssinica,Le genre Schefflera constitue un groupe d’arbr...
1,Schefflera Mannii,Schefflera mannii (Hook.f.) Harms est une espè...
2,Tessmannia africana,Tessmannia est un genre de plantes dicotylédon...
3,Alternanthera sessilis,Alternanthera est un genre qui regroupe plus d...
4,Pimpinella Ledermannii,Pimpinella ledermannii H. Wolff est une espèce...
...,...,...
141,Monopetalanthus Hedinii,Aphanocalyx hedinii est une espèce d'arbres de...
142,Monopetalanthus microphyllus,Monopetalanthus heitzii est une espèce de plan...
143,Deinbollia dasybotrys,Deinbollia est un genre de plantes de la famil...
144,Hydrocotyle sibthorpioides,Les hydrocotyles (Hydrocotyle) forment un genr...


In [4]:
data2

,espèce,description
0,Schefflera abyssinica,1. Schefflera abyssinica
1,Schefflera Mannii,3. Schefflera Mannii (Hooker fil.) Harms
2,Hydrocotyle sibthorpioides,1. Hydrocotyle sibthorpioides Lamarck
3,Hydrocotyle hirta,2. Hydrocotyle hirta R. Brown ex A. Richard
4,Hydrocotyle bonariensis,3. Hydrocotyle bonariensis Lamarck
...,...,...
200,Monopetalanthus Pellegrinii,5. Monopetalanthus Pellegrinii A. Chevalier
201,Swartzia fistuloides,1. Swartzia fistuloides Harms
202,Swartzia madagascariensis,2. Swartzia madagascariensis Desvaux
203,Csesalpinia Welwitschiana,1. Csesalpinia Welwitschiana (Oliver) Brenan


In [5]:
data=pd.concat([data1,data2]).reset_index(drop=True)

In [6]:
data

,espèce,description
0,Schefflera abyssinica,Le genre Schefflera constitue un groupe d’arbr...
1,Schefflera Mannii,Schefflera mannii (Hook.f.) Harms est une espè...
2,Tessmannia africana,Tessmannia est un genre de plantes dicotylédon...
3,Alternanthera sessilis,Alternanthera est un genre qui regroupe plus d...
4,Pimpinella Ledermannii,Pimpinella ledermannii H. Wolff est une espèce...
...,...,...
346,Monopetalanthus Pellegrinii,5. Monopetalanthus Pellegrinii A. Chevalier
347,Swartzia fistuloides,1. Swartzia fistuloides Harms
348,Swartzia madagascariensis,2. Swartzia madagascariensis Desvaux
349,Csesalpinia Welwitschiana,1. Csesalpinia Welwitschiana (Oliver) Brenan


In [7]:
import nltk
import random
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def TOKENIZE(Text):
    text=word_tokenize(Text)
    return(' '.join(text))
def tokenrandom(text,data,eps):
    seps=[',',';','\n',' ']
    i=-1
    if(random.uniform(0, 1)<eps):
        for i,row in data.iterrows():
            sep=random.choice(seps)
            text=row["description"]+sep+text
            data=data.drop(i)
            break
    return(TOKENIZE(text),data,i)
    

c:\Users\ATLAS PRO ELECTRO\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to C:\Users\ATLAS PRO
[nltk_data]     ELECTRO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import re
TRAIN_DATA=[]
df=data.copy()
for idx,row in data.iterrows():
    entities=[]
    if (row['description'].lower().find(row['espèce'].lower()))!=-1:
        description,df,i=tokenrandom(row['description'].lower().replace('.','').strip(),df,0.05)
    else:
        continue
    first_indexes1=[m.start() for m in re.finditer(row['espèce'].lower(), description)]
    for fr in first_indexes1:
        entities.append([fr,fr+len(row['espèce']),'ESPECE'])
    try:
        first_indexes2=[m.start() for m in re.finditer(data['espèce'][i].lower(), description)]
        for fr in first_indexes2:
            entities.append([fr,fr+len(data['espèce'][i]),'ESPECE'])

    except Exception as e:
        print(e)
    if entities:
        TRAIN_DATA.append([description,{"entities":entities}])

well the first problem that we faced with the other models we have created is that the model detects just one entitie a time so we need to detect multiple entities 

In [9]:
TRAIN_DATA1=TRAIN_DATA

after saving the data , we need to add more data that can helps to eal with the problem we had before

In [23]:
data3=pd.read_csv('../generetad data/rescued_data.csv')

In [24]:
#drop duplicates and empty rows
data3=data3.dropna()
data3=data3.drop_duplicates()
data3.reset_index(drop=True,inplace=True)
data3

,espèce,description
0,Aa achalensis,Aa est un genre de plante de la famille des Or...
1,Aa argyrolepis,Aa est un genre de plante de la famille des Or...
2,Aa aurantiaca,Echinopsis est l'un des principaux genres de l...
3,Aa colombiana,Aa est un genre de plante de la famille des Or...
4,Aa denticulata,Aa est un genre de plante de la famille des Or...
...,...,...
34786,Campanula macrostachya,Cette liste des plantes à feuilles comestibles...
34787,Campanula macrostyla,Les campanules (genre Campanula du latin campa...
34788,Campanula marchesettii,"Euphrasia, les Euphraises, est un genre de pla..."
34789,Campanula medium,La Campanule carillon (Campanula medium) est u...


In [25]:
data3=data3[data3['espèce'].str.match('^\w+\s[a-z]\w+$')].reset_index(drop=True)
data3

,espèce,description
0,Aa achalensis,Aa est un genre de plante de la famille des Or...
1,Aa argyrolepis,Aa est un genre de plante de la famille des Or...
2,Aa aurantiaca,Echinopsis est l'un des principaux genres de l...
3,Aa colombiana,Aa est un genre de plante de la famille des Or...
4,Aa denticulata,Aa est un genre de plante de la famille des Or...
...,...,...
34610,Campanula macrostachya,Cette liste des plantes à feuilles comestibles...
34611,Campanula macrostyla,Les campanules (genre Campanula du latin campa...
34612,Campanula marchesettii,"Euphrasia, les Euphraises, est un genre de pla..."
34613,Campanula medium,La Campanule carillon (Campanula medium) est u...


In [26]:
data3=data3.astype('string')
data3.dtypes

espèce         string
description    string
dtype: object

In [27]:
import nltk
import random
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def TOKENIZE(Text):
    text=word_tokenize(Text)[0:512]
    return(' '.join(text))
def tokenrandom(text,data,eps,esp_name):
    seps=[',',';','\n',' ']
    i=-1
    j=0
    if(random.uniform(0, 1)<eps):
        for i,row in data.iterrows():
            if (row['description'].lower().find(row['espèce'].lower()))!=-1 and row['espèce'].lower()!=esp_name:
                sep=random.choice(seps)
                text=row["description"]+sep+text
                data=data.drop(i)
                j=j+1
                if j>2:
                    raise ValueError("The function adds multiple sentences")
                break  
    return(TOKENIZE(text),data,i)
    

[nltk_data] Downloading package punkt to C:\Users\ATLAS PRO
[nltk_data]     ELECTRO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
#data3=data3.sample(5000).reset_index(drop=True)

In [29]:
data3

,espèce,description
0,Aa achalensis,Aa est un genre de plante de la famille des Or...
1,Aa argyrolepis,Aa est un genre de plante de la famille des Or...
2,Aa aurantiaca,Echinopsis est l'un des principaux genres de l...
3,Aa colombiana,Aa est un genre de plante de la famille des Or...
4,Aa denticulata,Aa est un genre de plante de la famille des Or...
...,...,...
34610,Campanula macrostachya,Cette liste des plantes à feuilles comestibles...
34611,Campanula macrostyla,Les campanules (genre Campanula du latin campa...
34612,Campanula marchesettii,"Euphrasia, les Euphraises, est un genre de pla..."
34613,Campanula medium,La Campanule carillon (Campanula medium) est u...


In [30]:
import re
TRAIN_DATA=[]
#TRAIN_DATA2=[]
df=data3.copy()
for idx,row in data3.iterrows():
    entities=[]
    if (row['description'].lower().find(row['espèce'].lower()))!=-1:
        description,df,i=tokenrandom(row['description'].lower().replace('.','').strip(),df,0.9,row['espèce'].lower())
    else:
        continue
    first_indexes1=[m.start() for m in re.finditer(row['espèce'].lower(), description)]
    for fr in first_indexes1:
        entities.append([fr,fr+len(row['espèce']),'ESPECE'])
    try:
        first_indexes2=[m.start() for m in re.finditer(data3['espèce'][i].lower(), description)]
        for fr in first_indexes2:
            entities.append([fr,fr+len(data3['espèce'][i]),'ESPECE'])
            print(i)
        #TRAIN_DATA2.append([description,{"entities":entities}])
    except Exception as e:
        i=-1
    if entities:
        TRAIN_DATA.append([description,{"entities":entities}])

8792
11485
11485
11485
12513
16449
17189
24932
25677


TRAIN_DATA2 is meant to collect the sentences that have more then one entitie 

In [31]:
len(TRAIN_DATA)


4719

In [32]:
TRAIN_DATA3=[]
for el in TRAIN_DATA:
    if(len(el[1]['entities'])>1):
        TRAIN_DATA3.append(el)

In [33]:
len(TRAIN_DATA3)

152

In [34]:
TRAIN_DATA3

[["L'Aibika , Abelmoschus manihot , est une espèce de plantes appartenant à la famille des Malvaceae , originaire du sud de la Chine , du Népal et de l'Inde . abies beshanzuensis ( en chinois 百山祖冷杉 baishanzu lengshan , littéralement « pin du baishanzu » ) est une espèce de sapins ( genre abies ) , endémique des monts baishanzu dans la province du zhejiang en république populaire de chine abies beshanzuensis se rencontre dans les forêts mixtes à atmosphère humide en chine du sud , à une altitude comprise entre 1 400 et 1 800 m d'altitude , dans une aire restreinte au nord-est de qingyuan abies beshanzuensis a été étudié et décrite pour la première fois en 1976 par le botaniste chinois mhwu en 2012 , il subsiste cinq individus adultes à l'état sauvage situés dans la réserve nationale naturelle de baishanzu l'union internationale pour la conservation de la nature la considère ainsi comme une « espèce en danger critique d'extinction » ( cr ) et l ' a intégrée à une liste des 100 espèces le

In [35]:
len(TRAIN_DATA1)

246

In [36]:
TRAIN_DATA1

[['schefflera mannii ( hookf ) harms est une espèce de plantes de la famille des araliaceae et du genre schefflera , endémique de la ligne montagneuse du cameroun',
  {'entities': [[0, 17, 'ESPECE']]}],
 ["Le genre Schefflera constitue un groupe d ’ arbres tropicaux vivaces au luxuriant feuillage persistant de la famille des Araliaceae , originaires de l ’ Asie et de l ’ Amérique centrale , où ils peuvent atteindre 30 m de haut . Au Queensland en Australie , il forme un petit arbre pouvant mesurer jusqu ’ à 12 m. En zone humide , ils poussent aussi en épiphytes sur des rochers ou des arbres . Il est dédié à Jacob Christian Scheffler , botaniste allemand du XIXe siècle . Le genre comprend environ 150 espèces et 900 variétés , originaires des zones tropicales et subtropicales humides , mais on cultive essentiellement les variétés arboricola et actinophylla aux feuilles vertes ou panachés de jaune ou d ’ ivoire , selon les hybrides , comme plantes tropicales d ’ intérieur , et peuvent att

In [37]:
#NOW we need to create a test set to evaluate our models on (20% of the original data)
TEST_DATA=TRAIN_DATA1[int(len(TRAIN_DATA1)*0.8):len(TRAIN_DATA1)]+TRAIN_DATA3[int(len(TRAIN_DATA3)*0.8):len(TRAIN_DATA3)]
len(TEST_DATA)

81

In [38]:
TRAIN_DATA=TRAIN_DATA1[0:int(len(TRAIN_DATA1)*0.8)]

In [39]:
for el in TRAIN_DATA3[0:int(len(TRAIN_DATA3)*0.8)]:
    TRAIN_DATA.append(el)

In [40]:
len(TRAIN_DATA)

317

In [42]:
import pickle
with open ("../merged data/train_species_data_merged.pkl","wb") as pick:
    pickle.dump(TRAIN_DATA,pick)
with open ("../merged data/test_species_data_merged.pkl","wb") as pick:
    pickle.dump(TEST_DATA,pick)

Generate train.spacy and test.spacy using the convert function 

In [45]:
TRAIN_DATA=[]
with open('train_species_data_merged.pkl','rb') as pick:
    TRAIN_DATA.append(pickle.load(pick))

TEST_DATA=[]
with open('test_species_data_merged.pkl','rb') as pick:
    TEST_DATA.append(pickle.load(pick))

In [44]:
from Convert_to_spacy_format import convert

In [47]:
convert("fr", TRAIN_DATA[0], "ner_data/train_species.spacy")
convert("fr", TRAIN_DATA[0], "ner_data/valid_species.spacy")
convert("fr",TEST_DATA[0], "ner_data/test_species.spacy")


e:\M2\NLP\NLP_project-master\Convert_to_spacy_format.py:20: UserWarning: Skipping entity [219, 232, ESPECE] in the following text because the character span 'allium cupani' does not align with token boundaries:

"Aira praecox , la canche précoce ou canche printanière , est une espèce végétale de la famille des Poaceae ( graminées ) . , allium cupani est une espèce d'oignon sauvage du centre et de l'est du région méditerranéenne allium cupanii forme un bulbe géophyte atteignant 0,3 m il fleurit en août l'espèce est hermaphrodite et entomophile il est adapté aux sols légers ( sableux ) et moyens et préfère les sols bien drainés"

  warnings.warn(msg)
e:\M2\NLP\NLP_project-master\Convert_to_spacy_format.py:20: UserWarning: Skipping entity [959, 978, ESPECE] in the following text because the character span 'allium platyspathum' does not align with token boundaries:

"Alchemilla glabra est une espèce de plantes de la famille des Rosaceae. , allium platyspathum est une espèce de plante bulbe